In [7]:
from edgedroid import data as e_data

data, *bins = e_data.load_default_exec_time_data()
data

In [23]:
%matplotlib inline

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_context("notebook")
sns.set_style("whitegrid")

from edgedroid.models import preprocess_data

for fade_dist in (None, 4, 8):
    proc_data = preprocess_data(
        data,
        *bins,
        transition_fade_distance=fade_dist,
    )

    proc_data["trans_dur"] = proc_data[["transition", "duration"]].apply(tuple, axis=1)
    proc_data["imp_trans"] = proc_data[["impairment", "transition"]].apply(tuple, axis=1)

    fg = sns.catplot(
        kind="point",
        data=proc_data,
        x="duration",
        hue="impairment",
        y="next_exec_time",
        dodge=True,
        col="transition",
    )
    fg.legend.set_title(f"{fade_dist}")
    for ax in fg.axes.flat:
        ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=15, ha="right", rotation_mode="anchor")

    plt.show()

In [90]:
import numpy as np
from collections import deque
from edgedroid.models.timings import ModelException, Transition
from typing import Optional
from pandas import arrays
import pandas as pd

proc_data, neuro_bins, impair_bins, duration_bins = e_data.load_default_exec_time_data()

ttfs = proc_data["ttf"].to_numpy()
ttfs = np.concatenate((ttfs, [-np.inf, np.inf]))

_, impair_bins = pd.qcut(proc_data["ttf"], 3, retbins=True)

impair_bins[0], impair_bins[-1] = -np.inf, np.inf

impair_bins = pd.arrays.IntervalArray.from_breaks(impair_bins, closed="left")

# impair_bins = pd.arrays.IntervalArray.from_breaks([-np.inf, 1, 3, 5, np.inf])

proc_data["neuroticism_raw"] = proc_data["neuroticism"]
proc_data["neuroticism"] = pd.cut(proc_data["neuroticism"], pd.IntervalIndex(neuro_bins))

processed_dfs = deque()
for run_id, df in proc_data.groupby("run_id"):
    # df = df.copy()
    df = df.copy()
    df["ttf"] = df["ttf"].shift().fillna(0)

    df["impairment"] = pd.cut(df["ttf"], pd.IntervalIndex(impair_bins))
    df = df.rename(columns={"exec_time": "next_exec_time"})

    # df["next_exec_time"] = df["exec_time"].shift(-1)
    df["prev_impairment"] = df["impairment"].shift()
    # df["transition"] = Transition.NONE.value

    # for each segment with the same impairment, count the number of steps
    # (starting from 1)
    diff_imp_groups = df.groupby(
        (df["impairment"].ne(df["prev_impairment"])).cumsum()
    )
    df["duration"] = diff_imp_groups.cumcount() + 1

    df["transition"] = None
    df.loc[df["prev_impairment"] < df["impairment"], "transition"] = Transition.L2H.value
    df.loc[df["prev_impairment"] > df["impairment"], "transition"] = Transition.H2L.value

    df["transition"] = df["transition"].fillna(method="ffill")

    processed_dfs.append(df)

proc_data = pd.concat(processed_dfs, ignore_index=False)

# coerce some types for proper functionality
proc_data["transition"] = proc_data["transition"].astype("category")
proc_data["neuroticism"] = proc_data["neuroticism"].astype(pd.IntervalDtype(float))
proc_data["impairment"] = proc_data["impairment"].astype(pd.IntervalDtype(float))
proc_data["duration_raw"] = proc_data["duration"]
proc_data["duration"] = pd.cut(proc_data["duration"], pd.IntervalIndex(duration_bins)).astype(
    pd.IntervalDtype(float)
)
proc_data = proc_data.drop(columns="prev_impairment")

proc_data

In [97]:
fg = sns.catplot(
    kind="point",
    data=proc_data,
    x="duration",
    hue="impairment",
    y="next_exec_time",
    dodge=True,
    col="transition",
    row="neuroticism",
)
# for ax in fg.axes.flat:
#     ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=15, ha="right", rotation_mode="anchor")

plt.show()

In [28]:
fg = sns.catplot(
    kind="bar",
    data=proc_data[proc_data.duration == proc_data.duration.min()],
    y="next_exec_time",
    x="impairment",
    hue="transition"
)
for ax in fg.axes.flat:
    ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=15, ha="right", rotation_mode="anchor")

plt.show()